In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ROOT
import glob
import sys
import os
import tqdm

Welcome to JupyROOT 6.24/00


In [2]:
data_dir = '../build/data/'

# number of primaries for given run
# if not in this dict, assume files have 5e7 primaries
# (distributed between all threads)
primary_dict = {
    'simdata_20241231_121236': 1e7,
    'simdata_20241231_145727': 5e6
}

# dict to hold [file path -- thread counts]
data_files_found = {}

filenames = os.listdir(data_dir)

Nprimaries = 0. # progressively add to this as more files read

for fname in filenames:
    fbase = fname.split('_t')[0]
    idx = int(fname.split('_t')[1].split('.')[0])

    if fbase not in data_files_found:
        data_files_found[fbase] = []
    
    data_files_found[fbase].append(idx)

for fbase, idxs in data_files_found.items():
    if all(i in idxs for i in range(max(idxs)+1)):
        numfiles = len(idxs)
        if numfiles != 8:
            print(fbase, ': numfiles =', numfiles)
    else:
        print(fbase, ': not all found')
    if fbase in primary_dict:
        Nprimaries += primary_dict[fbase]
        #print('found', fbase)
    else:
        Nprimaries += 5e7

print(f'total {Nprimaries:.4g} primaries')

simdata_20241231_121236 : numfiles = 4
simdata_20241231_145727 : numfiles = 4
simdata_20241231_155127 : numfiles = 4
total 2.665e+09 primaries


In [3]:
Emin = 0.5e-6
Emax = 20 # MeV
Nbins = 100


Ebins = np.geomspace(Emin, Emax, Nbins+1, endpoint = False) # bin edges (MeV)
dE = (Ebins[1:] - Ebins[:-1]).mean()
Emids = Ebins + dE/2 # bin centers


detectors = ['lowmass', 'highmass1', 'highmass2'] # detectors
codetectors = ['lowmassxhighmass1', 'lowmassxhighmass2', 'highmass1xhighmass2', 'lowmassxhighmass1xhighmass2'] # coincidences
Qs = ['NR', 'ER', 'Ep', 'Eq'] # recoil types; Ep = total phonon ER+NR, Eq = total charge ER+Lindhard(NR)

Ncaptures = {det: 0 for det in detectors} # number of neutron captures in each detector
Nhits = {det: 0 for det in detectors} # total number of entries in trees


# all hits
# all_events[q][d] = histogram of events (true energy) of q recoils in detector d
all_events = {Q: {det: ROOT.TH1D(
        det + '_evts_' + Q, 
        det + ' all ' + Q + ' events',
        Nbins, Ebins
        ) for det in detectors} for Q in Qs}

# hits in coincidence with a hit in the lowmass
# coincidence_events[axb][c][q] = histogram of events in detector c that have coincidence in a & b (c must be either a or b) of recoil type q
coincidence_events = {detdet: {det: {Q: ROOT.TH1D(
            det + '_coinc_' + detdet , 
            detdet + ' coincidence in ' + det + ' ' + Q + ' events ',
            Nbins, Ebins
        ) for Q in Qs
    } for det in detdet.split('x')
} for detdet in codetectors}

# postcapture gammas 
# postcapture_gammas[a][b] -- parent nCapture in a, gamma hit in b -- individual gammas, not event by event
postcapture_gammas = {d1: {d2:  ROOT.TH1D(
        d1 + '-' + d2 + '_pcg', 
        d1 + ' --> ' + d2 + ' postcapture gammas',
        Nbins, Ebins
        ) for d2 in detectors} for d1 in detectors}


Warning in <TROOT::Append>: Replacing existing TH1: lowmass_coinc_lowmassxhighmass1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: lowmass_coinc_lowmassxhighmass1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: lowmass_coinc_lowmassxhighmass1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: highmass1_coinc_lowmassxhighmass1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: highmass1_coinc_lowmassxhighmass1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: highmass1_coinc_lowmassxhighmass1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: lowmass_coinc_lowmassxhighmass2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: lowmass_coinc_lowmassxhighmass2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: lowmass_coinc_lowmassxhighmass2 (Potential memory leak).
Warning in <T

In [4]:
def dcsd_to_string(tree, k):
    tree.GetEntry(k)
    EventNum = int(getattr(tree, 'EventNum'))
    TrkNum = int(getattr(tree, 'TrkNum'))
    Edep = float(getattr(tree, 'Edep'))
    VolName = getattr(tree, 'VolName')
    PName = getattr(tree, 'PName')
    ProcName = getattr(tree, 'ProcName')
    ParentVol = getattr(tree, 'ParentVol')
    IsCapture = getattr(tree, 'IsCapture')

    s = f"""
    EventNum        = {EventNum}
    TrkNum          = {TrkNum}
    Edep            = {Edep} [MeV]
    VolName         = {VolName}
    PName           = {PName}
    ProcName        = {ProcName}
    ParentVol       = {ParentVol}
    IsCapture       = {IsCapture}
    """

    return s

# most NRs should be silicon, but we also have some:

# most ERs should be e+/- and gammas, but we also have some
#   Al from neutron-induced proton knockout
#   protons from ^
#   P31 from beta decay of Si31 after nCapture on Si30
#   Mg25 (and so one) from neutron spallation of alphas
#   alphas from ^
#   deuterons from some bullshit

def get_evt_type(PName):
    if PName in 'e- e+ gamma proton P31 alpha deuteron' or PName[:2] in 'Al Mg':
        return 'ER'
    elif PName[:2] in 'Si ': 
        return 'NR'
    else:
        raise Exception('unknown PName = ' + PName + ', ProcName = ' + getattr(tree, 'ProcName'))


def yLind(Er):
    Er *= 1e3 # convert to keV from MeV
    Z = 28
    k = 0.15
    eps = 11.5*Er/np.cbrt(Z)**7
    g = 3*eps**0.15 + 0.7*eps**0.6 + eps
    Y = k*g/(1+k*g) # yield
    Eee = Y*Er/1e3 # electron equivalent energy (back to MeV)
    #return Eee*(4/3.8) # 4 Volts divided by 3.8 eV per e/h pair
    return Y # very low bias; charge channel readout 

In [5]:
for det in detectors:
    for Q in Qs:
        all_events[Q][det].Reset()

for detdet in codetectors:
    for det in detdet.split('x'):
        for Q in Qs:
            coincidence_events[detdet][det][Q].Reset()

for d1 in detectors:
    for d2 in detectors:
        postcapture_gammas[d1][d2].Reset()

lowmass_captures = {}

for file in tqdm.tqdm(filenames):
    #print(file)

    last_event = -1

    Edep_evt = {Q: {det: 0. for det in detectors} for Q in Qs} # Edep in each detector
    
    # open file
    tfile = ROOT.TFile.Open(data_dir + file, 'READ')
    tree = tfile.Get('tree')
    N = tree.GetEntries()
    n = tree.GetEntries('IsCapture') # number of captures in lowmass

    N = tree.GetEntries()

    
    for k in range(N):

        tree.GetEntry(k)

        EventNum = int(getattr(tree, 'EventNum'))
        TrkNum = int(getattr(tree, 'TrkNum'))
        Edep = float(getattr(tree, 'Edep'))
        VolName = getattr(tree, 'VolName')
        PName = getattr(tree, 'PName')
        ProcName = getattr(tree, 'ProcName')
        ParentVol = getattr(tree, 'ParentVol')
        IsCapture = int(getattr(tree, 'IsCapture'))

        q = get_evt_type(PName)

        Nhits[VolName] += 1
        if (ProcName == 'nCapture') and (ParentVol in detectors):
            Ncaptures[ParentVol] += 1
            if ParentVol == 'lowmass':
                if file not in lowmass_captures:
                    lowmass_captures[file] = [EventNum]
                elif EventNum not in lowmass_captures[file]:
                    lowmass_captures[file].append(EventNum)



        if abs(EventNum - last_event) > 1e-6: # if new event
            last_event = EventNum 

            # list of detectors where there were hits
            yes_hit = [det for det in detectors if any(Edep_evt[Q][det] > 0 for Q in Qs)]

            co = ''
            if len(yes_hit) > 1:
                co = 'x'.join(yes_hit)
            
            # fill data into histograms
            # Edep_evt[det] holds the total energy [MeV] from the event
            for det in detectors:
                # fill histograms 
                for Q in Qs:
                    all_events[Q][det].Fill(Edep_evt[Q][det])

                    if co and (det in co):
                        coincidence_events[co][det][Q].Fill(Edep_evt[Q][det])


            # reset variables
            for det in detectors:
                for Q in Qs:

                    Edep_evt[Q][det] = 0.
           
        
        # all events: add current Edep (of the correct type) to the totals

        Edep_evt[q][VolName] += Edep # add to corresponding event type

        # charge channels: NR get lindhard yield
        # phonon channels: Er plus charge channels times V/eps
        # assume 4 volts
        V = 4
        eps = 3.8
        if q == 'ER':
            Eq = Edep
        elif q == 'NR':
            Eq = Edep*yLind(Edep)

        Edep_evt['Eq'][VolName] += Eq
        Edep_evt['Ep'][VolName] += Edep + Eq*V/eps

        # hit by hit histograms

        # nCapture gammas:
        if ProcName == 'nCapture' and PName == 'gamma' and ParentVol in detectors:
            postcapture_gammas[ParentVol][VolName].Fill(Edep)
        
        
    tfile.Close()


for det in detectors:
    print(f'{det}: {Nhits[det]:.3g} hits total, {Ncaptures[det]} capture events in {det}')

100%|██████████| 428/428 [07:04<00:00,  1.01it/s]

lowmass: 1.15e+04 hits total, 6 capture events in lowmass
highmass1: 5.81e+06 hits total, 10860 capture events in highmass1
highmass2: 3.98e+06 hits total, 7502 capture events in highmass2


In [6]:
lowmass_captures

{'simdata_20241231_155127_t2.root': [13803313],
 'simdata_20241231_171952_t0.root': [20715467],
 'simdata_20241231_181106_t3.root': [7557213],
 'simdata_20250101_023645_t7.root': [7530268],
 'simdata_20250101_052835_t1.root': [49174986],
 'simdata_20250101_062002_t7.root': [11038245]}

In [7]:
all_events['NR']['lowmass'].GetName()

'lowmass_evts_NR'

In [8]:
# array to hold data
cts = np.zeros(Nbins+1)

# all hits
for Q in Qs:
    for det in detectors:
        hist = all_events[Q][det]
        name = hist.GetName()
        
        for i in range(1, Nbins+1):
            cts[i-1] = hist.GetBinContent(i)
        
        df = pd.DataFrame.from_dict({'E': Ebins, 'count': cts})
        df.to_csv('analysis_data/' + name + '.txt', index = False)



# coincidences
for detdet in codetectors:
    for det in detdet.split('x'):
        for Q in Qs:
            hist = coincidence_events[detdet][det][Q]
            name = hist.GetName()
            
            for i in range(1, Nbins+1):
                cts[i-1] = hist.GetBinContent(i)
            
            df = pd.DataFrame.from_dict({'E': Ebins, 'count': cts})
            df.to_csv('analysis_data/' + name + '.txt', index = False)


# postcapture gammas
for d1 in detectors:
    for d2 in detectors:
        hist = postcapture_gammas[d1][d2]

        name = hist.GetName()
        
        for i in range(1, Nbins+1):
            cts[i-1] = hist.GetBinContent(i)
        
        df = pd.DataFrame.from_dict({'E': Ebins, 'count': cts})
        df.to_csv('analysis_data/' + name + '.txt', index = False)



In [9]:
coincidence_events.keys()

dict_keys(['lowmassxhighmass1', 'lowmassxhighmass2', 'highmass1xhighmass2', 'lowmassxhighmass1xhighmass2'])

In [10]:
hist = all_events['Ep']['highmass1']

for i, E in enumerate(Ebins):
    E1 = hist.GetBinLowEdge(i+1)

    print(E, E1)

5e-07 5e-07
5.94617826409102e-07 5.94617826409102e-07
7.071407189669702e-07 7.071407189669702e-07
8.40956954555021e-07 8.40956954555021e-07
1.0000959928422494e-06 1.0000959928422494e-06
1.1893498109286226e-06 1.1893498109286226e-06
1.4144171988289084e-06 1.4144171988289084e-06
1.6820753608065964e-06 1.6820753608065964e-06
2.000383989798249e-06 2.000383989798249e-06
2.378927959994805e-06 2.378927959994805e-06
2.8291059455119014e-06 2.8291059455119014e-06
3.3644736560027163e-06 3.3644736560027163e-06
4.001152024686041e-06 4.001152024686041e-06
4.758312640102384e-06 4.758312640102384e-06
5.658755038865272e-06 5.658755038865272e-06
6.729593242783259e-06 6.729593242783259e-06
8.003072213282325e-06 8.003072213282325e-06
9.517538808114039e-06 9.517538808114039e-06
1.1318596477690093e-05 1.1318596477690093e-05
1.3460478471131607e-05 1.3460478471131607e-05
1.6007680901861615e-05 1.6007680901861615e-05
1.90369048474309e-05 1.90369048474309e-05
2.2639365963872523e-05 2.2639365963872523e-05
2.6923